In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1003. Ozone Sterols Healthy Asthma/P1003.3. Analyses/P1003.3.2. Ozone Responder Prediction/Input')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1003. Ozone Sterols Healthy Asthma/P1003.3. Analyses/P1003.3.2. Ozone Responder Prediction/Output')
cur_date = "090822"

library(readxl)
library(tidyverse)
library(gtsummary)
library(caret)
library(e1071)
library(Hmisc)
library(glmnet)
library(randomForest)
library(pROC)


#reading in files
ozone_data = data.frame(read_excel("Imputed_Ozone_Data_042622.xlsx")) 
demographics_data = data.frame(read_excel("SubjectInfo_011822.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

#BlackLivesMatter

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Loading required package: survival


Attaching package: ‘survival’


The following object is masked from ‘package:caret’:

    cluster


Loading required package: Formula


Attaching package: ‘Hmisc’


The following object is masked from ‘package:e1071’:

    impute


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


Loading required package: Matrix


Atta

In [2]:
head(ozone_data)
head(demographics_data)

,Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,Sample_Weight,817.00
2,4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,TCC,395000.00
3,4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,TCC_Weight,483.48
4,4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,Neutrophils_Percent,25.00
5,4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,Neutrophils,98750.00
6,4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,Neutrophils_Weight,121.00


,Original_Subject_No,Subject_No,Subject_ID,Sex,Race,Ethnicity,Age,BMI,Disease_Status,Visit_Season
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
1,4,1,AS_1,M,W,NH,30,23.3,AS,Fa
2,7,2,AS_2,F,W,NH,39,32.7,AS,Sp
3,11,3,AS_3,M,W,NH,23,23.0,AS,Fa
4,13,4,AS_4,M,W,NH,20,21.4,AS,Wi
5,15,5,AS_5,F,B,NH,20,34.5,AS,Su
6,16,6,AS_6,F,W,NH,25,25.2,AS,Su


In [3]:
plasma_baseline_ozone_data = full_join(demographics_data, ozone_data) %>%
    # only interested in plasma soluble mediators (sterols) at baseline
    filter(Sample_Type == "P" & Time_Point == "Pre") %>%
    filter(Category == "Sterol") %>%
    select(-c("Sample_Type", "Time_Point", "Category")) %>%
    distinct()
head(plasma_baseline_ozone_data)

Joining, by = c("Original_Subject_No", "Subject_No", "Subject_ID",
"Disease_Status")


,Original_Subject_No,Subject_No,Subject_ID,Sex,Race,Ethnicity,Age,BMI,Disease_Status,Visit_Season,Inflammatory_Responder,Lung_Responder,Variable,Value
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,4,1,AS_1,M,W,NH,30,23.3,AS,Fa,NR,NR,25OH_VitD,130.325
2,4,1,AS_1,M,W,NH,30,23.3,AS,Fa,NR,NR,25OH_VitD,47.181
3,4,1,AS_1,M,W,NH,30,23.3,AS,Fa,NR,NR,SecoB,1.816
4,4,1,AS_1,M,W,NH,30,23.3,AS,Fa,NR,NR,25OH_VitD,16.537
5,4,1,AS_1,M,W,NH,30,23.3,AS,Fa,NR,NR,25OH_VitD,15.326
6,4,1,AS_1,M,W,NH,30,23.3,AS,Fa,NR,NR,25OH_VitD,280.856


Seeing if there are statistical differences in predictors stratified by inflammatory or lung status.

In [4]:
# first pivoting wider to make the tbl_summary function work
wider_data = pivot_wider(plasma_baseline_ozone_data,
                         #not actually taking the mean of multiple values
                         #did this bc cols would be type list rather than double
                         names_from = Variable, values_from = Value, values_fn = mean)

head(wider_data)

Original_Subject_No,Subject_No,Subject_ID,Sex,Race,Ethnicity,Age,BMI,Disease_Status,Visit_Season,⋯,SecoB,bEP_Chol,Lan,Zym,DHL,Des,dHLan,Zyme,Lath,Chol
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4,1,AS_1,M,W,NH,30,23.3,AS,Fa,⋯,1.816,25.669,0.171,0.057,0.224,0.641,0.004,3.014,1.969,1110.442
7,2,AS_2,F,W,NH,39,32.7,AS,Sp,⋯,112.521,17.867,0.062,0.046,0.106,0.286,0.015,2.508,1.287,673.328
11,3,AS_3,M,W,NH,23,23.0,AS,Fa,⋯,6.686,8.066,0.052,0.046,0.094,0.279,0.005,1.549,1.200,537.509
13,4,AS_4,M,W,NH,20,21.4,AS,Wi,⋯,48.416,10.132,0.013,0.046,0.085,0.297,0.016,1.833,1.062,482.396
15,5,AS_5,F,B,NH,20,34.5,AS,Su,⋯,13.006,13.090,0.117,0.060,0.116,0.255,0.020,3.628,1.795,586.750
16,6,AS_6,F,W,NH,25,25.2,AS,Su,⋯,36.458,14.331,0.022,0.034,0.077,0.272,0.006,1.914,0.966,639.636


In [5]:
#summary statistics df
wider_data %>%
  tbl_summary(by = Inflammatory_Responder, missing = "no", 
  include = colnames(wider_data[13:23]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% #adding p value from anova
  as_tibble()

**Characteristic**,**N**,"**NR**, N = 8","**R**, N = 16",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
25OH_VitD,24,53 (12),50 (14),0.5
SecoB,24,18 (20),38 (35),0.14
bEP_Chol,24,15.7 (7.1),11.7 (4.4),0.11
Lan,24,0.09 (0.08),0.08 (0.08),0.7
Zym,24,0.057 (0.040),0.045 (0.025),0.4
DHL,24,0.14 (0.13),0.10 (0.05),0.2
Des,24,0.37 (0.25),0.26 (0.08),0.10
dHLan,24,0.007 (0.008),0.007 (0.006),0.9
Zyme,24,2.32 (1.39),2.14 (1.08),0.7


In [6]:
wider_data %>%
  tbl_summary(by = Lung_Responder, missing = "no", 
  include = colnames(wider_data[13:23]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% #adding p value from anova
  as_tibble()

**Characteristic**,**N**,"**NR**, N = 12","**R**, N = 12",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
25OH_VitD,24,46 (15),56 (8),0.065
SecoB,24,33 (31),29 (34),0.8
bEP_Chol,24,15.1 (6.3),11.0 (4.1),0.072
Lan,24,0.11 (0.09),0.06 (0.06),0.12
Zym,24,0.065 (0.035),0.033 (0.011),0.005
DHL,24,0.15 (0.10),0.08 (0.04),0.040
Des,24,0.36 (0.20),0.23 (0.07),0.037
dHLan,24,0.011 (0.007),0.003 (0.003),0.002
Zyme,24,2.84 (1.29),1.57 (0.54),0.005


These p values are relatively high, so I'm not expecting the model to predict well. 

# Which demographic information and/or soluble mediators in plasma collected at baseline best predict ozone-responder status in asthmatics and non-asthmatics?

This question will be tackled using 3 supervised machine learning methods assessing and classification...

1. K Nearest Neighbor (KNN)
2. Random Forest
3. Support Vector Machine (SVM)

In [7]:
#making responder columns binary denoted by 0 = non-responder and 1 = responder and covariates
wider_data = wider_data %>%
    #select(-DHL) %>%
    mutate(Inflammatory_Responder = relevel(factor(ifelse(Inflammatory_Responder == "R", 1, 0)), ref = "0")) %>%
    mutate(Lung_Responder = relevel(factor(ifelse(Lung_Responder == "R", 1, 0)), ref = "0")) %>%
    # for some reason RF didn't this variable so I changed it
    rename(X25OH_VitD = `25OH_VitD`) %>%
    mutate(Sex = relevel(factor(ifelse(Sex == "M", 1, 0)), ref = "0")) %>%
    mutate(Race = relevel(factor(ifelse(Race == "M", 1,
                                        ifelse(Race == "As", 2,
                                        ifelse(Race == "B", 3, 0)))), ref = "0")) %>%
    mutate(Ethnicity = relevel(factor(ifelse(Ethnicity == "H", 1, 0)), ref = "0")) %>%
    mutate(Disease_Status = relevel(factor(ifelse(Disease_Status == "AS", 1, 0)), ref = "0")) %>%
    mutate(Visit_Season = relevel(factor(ifelse(Visit_Season == "Su", 3, 
                                               ifelse(Visit_Season == "Sp", 2,
                                               ifelse(Visit_Season == "Fa", 1, 0)))), ref = "0"))

head(wider_data)

Original_Subject_No,Subject_No,Subject_ID,Sex,Race,Ethnicity,Age,BMI,Disease_Status,Visit_Season,⋯,SecoB,bEP_Chol,Lan,Zym,DHL,Des,dHLan,Zyme,Lath,Chol
<dbl>,<dbl>,<chr>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4,1,AS_1,1,0,0,30,23.3,1,1,⋯,1.816,25.669,0.171,0.057,0.224,0.641,0.004,3.014,1.969,1110.442
7,2,AS_2,0,0,0,39,32.7,1,2,⋯,112.521,17.867,0.062,0.046,0.106,0.286,0.015,2.508,1.287,673.328
11,3,AS_3,1,0,0,23,23.0,1,1,⋯,6.686,8.066,0.052,0.046,0.094,0.279,0.005,1.549,1.200,537.509
13,4,AS_4,1,0,0,20,21.4,1,0,⋯,48.416,10.132,0.013,0.046,0.085,0.297,0.016,1.833,1.062,482.396
15,5,AS_5,0,3,0,20,34.5,1,3,⋯,13.006,13.090,0.117,0.060,0.116,0.255,0.020,3.628,1.795,586.750
16,6,AS_6,0,0,0,25,25.2,1,3,⋯,36.458,14.331,0.022,0.034,0.077,0.272,0.006,1.914,0.966,639.636


In [8]:
inflamm_resp_df = wider_data[,11:23] %>% select(-Lung_Responder)
inflamm_resp_covariates_df = wider_data[,4:23] %>% select(-Lung_Responder)
lung_resp_df = wider_data[,12:23]
lung_resp_covariates_df = wider_data[,4:23] %>% select(-Inflammatory_Responder)

head(lung_resp_covariates_df)

Sex,Race,Ethnicity,Age,BMI,Disease_Status,Visit_Season,Lung_Responder,X25OH_VitD,SecoB,bEP_Chol,Lan,Zym,DHL,Des,dHLan,Zyme,Lath,Chol
<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,0,30,23.3,1,1,0,61.57312,1.816,25.669,0.171,0.057,0.224,0.641,0.004,3.014,1.969,1110.442
0,0,0,39,32.7,1,2,0,20.31488,112.521,17.867,0.062,0.046,0.106,0.286,0.015,2.508,1.287,673.328
1,0,0,23,23.0,1,1,1,50.92712,6.686,8.066,0.052,0.046,0.094,0.279,0.005,1.549,1.200,537.509
1,0,0,20,21.4,1,0,0,30.55712,48.416,10.132,0.013,0.046,0.085,0.297,0.016,1.833,1.062,482.396
0,3,0,20,34.5,1,3,0,29.90213,13.006,13.090,0.117,0.060,0.116,0.255,0.020,3.628,1.795,586.750
0,0,0,25,25.2,1,3,0,33.79675,36.458,14.331,0.022,0.034,0.077,0.272,0.006,1.914,0.966,639.636


# Random Forest
- an ensemble learning method operating by constructing a multitude of decision trees at training time, which uses multiple methods to obtain a better predictive performance and includes bagging and random forest
- algorithm uses a bootstrop aggregation, to reduce overfitting the training datset but only a subset of the features are used hence decorrelation of predictors

In [9]:
rf_classification = function(dataset, outcome, pred_outcome){
    # setting for reproducibility
    set.seed(12)
    # splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) # 5 fold CV

    metrics = data.frame()
    variable_importance_df = data.frame()
    roc_objects = c()
    threshold_data = data.frame()
    
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        ntree_values = c(50, 250, 500) # number of trees (not sure if we wanted something different)
        p = dim(dataset)[2] - 1 # number of variables in dataset
        mtry_values = c(sqrt(p), p/2, p/3)

        # will use ntree and mtry values to determine which combination yields the smallest MSE
        reg_rf_pred_tune = list()
        rf_OOB_errors = list()
        rf_error_df = data.frame()
        for (j in 1:length(ntree_values)){
            for (k in 1:length(mtry_values)){
                reg_rf_pred_tune[[k]] = randomForest(as.formula(paste0(outcome, "~.")), data = data_train, 
                                                     ntree = ntree_values[j], mtry = mtry_values[k])
                rf_OOB_errors[[k]] = data.frame("Tree Number" = ntree_values[j], "Variable Number" = mtry_values[k], 
                                       "OOB_errors" = reg_rf_pred_tune[[k]]$err.rate[ntree_values[j],1])
                rf_error_df = rbind(rf_error_df, rf_OOB_errors[[k]])
            }
        }

        # finding the lowest OOB error using best number of predictors at split and refitting OG tree
        best_oob_errors <- which(rf_error_df$OOB_errors == min(rf_error_df$OOB_errors))
        
        # many models have the lowest errors, so now selecting based on largest number of trees grown
        best_oob_df = rf_error_df[best_oob_errors, ]
        largest_trees = which(best_oob_df$Tree.Number == max(best_oob_df$Tree.Number))
        
        # still duplicate models w/ the lowest errors, so now selecting based on # of predictors = sqrt(p)
        best_tree_df = best_oob_df[largest_trees, ]
        default_predictor_number = which(best_tree_df$Variable.Number == max(best_tree_df$Variable.Number))


        reg_rf <- randomForest(as.formula(paste0(outcome, "~.")), data = data_train,
                               ntree = best_tree_df$Tree.Number[default_predictor_number],
                               mtry = best_tree_df$Variable.Number[default_predictor_number])

        # predicting on test set
        data_test[[pred_outcome]] = predict(reg_rf, newdata = data_test, type = "response")
        
        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                     positive = "1")

        # calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))

        
        # calculating values to plot ROC curve later
        roc_obj = roc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))

        # Return max Youden's index, with specificity and sensitivity
        best_thres_data = data.frame(coords(roc_obj, x = "best", best.method = c("youden", "closest.topleft")))
        threshold_data = rbind(threshold_data, best_thres_data)
        
        # extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
        
        # extracting variable importance
        var_importance_values = data.frame(importance(reg_rf)) %>%
            rownames_to_column(var = "Predictor")
        variable_importance_df = rbind(variable_importance_df, var_importance_values)
   
        # adding values to df
        metrics = rbind(metrics, matrix_values)
        
    }
    
    # taking averages/sd 
    metrics = metrics %>%
        summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
    
    variable_importance_df = variable_importance_df %>%
        group_by(Predictor) %>%
        summarise(MeanDecreaseGini = mean(MeanDecreaseGini)) %>%
        # sorting by most important variables
        arrange(-MeanDecreaseGini)
  
    # return training set, matrix, variable importance values, (last) roc object, best threshold data
    return(list(data_train, metrics, variable_importance_df, roc_obj, threshold_data))

}

In [10]:
# calling fn
rf_values_inflamm_resp = rf_classification(inflamm_resp_df, "Inflammatory_Responder", "pred_Inflammatory_Responder")
rf_values_lung_resp = rf_classification(lung_resp_df, "Lung_Responder", "pred_Lung_Responder")
rf_values_inflamm_resp_covariates = rf_classification(inflamm_resp_covariates_df, "Inflammatory_Responder", "pred_Inflammatory_Responder")
rf_values_lung_resp_covariates = rf_classification(lung_resp_covariates_df, "Lung_Responder", "pred_Lung_Responder")

Setting levels: control = 0, case = 1

Setting direction: controls > cases

Setting levels: control = 0, case = 1

Setting direction: controls > cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [11]:
# viewing results
rf_confusion_matrix = data.frame(Model = c(rep(c("RF Classification"), times = 4)), 
                                 Response = c(rep(c("Inflammatory"), times = 2), rep(c("Lung"), times = 2)),
                                 Covariates = c(rep(c("No", "Yes"), times = 2)), Kernel = c(rep(c("NA"), times = 4)), rbind(rf_values_inflamm_resp[[2]], 
                                rf_values_inflamm_resp_covariates[[2]], rf_values_lung_resp[[2]], rf_values_lung_resp_covariates[[2]]))
rf_confusion_matrix

# viewing most significant features
head(rf_values_inflamm_resp[[3]])
head(rf_values_inflamm_resp_covariates[[3]])
head(rf_values_lung_resp[[3]]) 
head(rf_values_lung_resp_covariates[[3]]) 

Model,Response,Covariates,Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF Classification,Inflammatory,No,NA,0.4083333,0.8166667,0.0000000,0.6100000,0.5416667
RF Classification,Inflammatory,Yes,NA,0.3583333,0.7166667,0.0000000,0.5733333,0.5083333
RF Classification,Lung,No,NA,0.8500000,0.8333333,0.8666667,0.8666667,0.8500000
RF Classification,Lung,Yes,NA,0.8500000,0.8333333,0.8666667,0.8666667,0.8500000


Predictor,MeanDecreaseGini
<chr>,<dbl>
Chol,1.1747072
Lath,1.0983830
bEP_Chol,1.0255966
SecoB,0.9204824
Des,0.7188831
Zyme,0.5987638


Predictor,MeanDecreaseGini
<chr>,<dbl>
bEP_Chol,0.9616116
Lath,0.8531715
Chol,0.7928745
SecoB,0.7683006
Zyme,0.5945749
Des,0.5937323


Predictor,MeanDecreaseGini
<chr>,<dbl>
Chol,1.9653682
dHLan,1.5438321
Zym,1.3533973
X25OH_VitD,0.8161816
DHL,0.7516207
Zyme,0.7339622


Predictor,MeanDecreaseGini
<chr>,<dbl>
Chol,1.9791634
dHLan,1.3501283
Zym,1.2128085
X25OH_VitD,0.6894127
DHL,0.6823232
Zyme,0.6125265


# Support Vector Machine (SVM)
- supervised learning models that can predict continuous (regression) or grouped/dichotomous (classification) outcomes
- predicts by projecting predictors onto a high dimensional space and uses kernels to make the data more separable (unfortunately makes interpretability of model results more difficult)
- does a better job at handling a large number of predictors since p > n

In [12]:
# model errors are calculated to assess the predictive accuracy of the model
svm_classification = function(dataset, model, outcome, pred_outcome, gamma_values, cost_values, elsilon_values){
  
    # setting seed for reproducibility
    set.seed(12)
    
    # splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV
    metrics = data.frame()
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        # now pruning parameters (based on the training dataset to prevent overfitting)
        svr_tune <- tune(svm, as.formula(paste0(outcome, "~.")), data = data_train, kernel = model, 
                       ranges = list(elsilon = elsilon_values, cost = cost_values, gamma = gamma_values)) 

        # choosing best model
        best_svm <- svr_tune$best.model

        # predicting with tuned parameters 
        data_test[[pred_outcome]] <- predict(best_svm, newdata = data_test, type = "response")

        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                 positive = "1")
 
        # calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        # extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
   
        # adding values to df
        metrics = rbind(metrics, matrix_values)
  }
  
  # taking averages/sd 
  metrics = metrics %>%
    summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
  
  return(metrics)
}

In [13]:
# calling fn
# linear kernel
svm_linear_inflamm = svm_classification(inflamm_resp_df, "linear", "Inflammatory_Responder", "pred_Inflammatory_Responder",
                                             0.035, 1:5, seq(0,1,0.2))
svm_linear_lung = svm_classification(lung_resp_df, "linear", "Lung_Responder", "pred_Lung_Responder",
                                             0.035, 1:5, seq(0,1,0.2))
svm_linear_inflamm_covariates = svm_classification(inflamm_resp_covariates_df, "linear", "Inflammatory_Responder", "pred_Inflammatory_Responder",
                                             0.035, 1:5, seq(0,1,0.2))
svm_linear_lung_covariates = svm_classification(lung_resp_covariates_df, "linear", "Lung_Responder", "pred_Lung_Responder",
                                             0.035, 1:5, seq(0,1,0.2))
# radial
svm_radial_inflamm = svm_classification(inflamm_resp_df, "radial", "Inflammatory_Responder", "pred_Inflammatory_Responder",
                                             0.035, 1:5, seq(0,1,0.2))
svm_radial_lung = svm_classification(lung_resp_df, "radial", "Lung_Responder", "pred_Lung_Responder",
                                             0.035, 1:5, seq(0,1,0.2))
svm_radial_inflamm_covariates = svm_classification(inflamm_resp_covariates_df, "radial", "Inflammatory_Responder", "pred_Inflammatory_Responder",
                                             0.035, 1:5, seq(0,1,0.2))
svm_radial_lung_covariates = svm_classification(lung_resp_covariates_df, "radial", "Lung_Responder", "pred_Lung_Responder",
                                             0.035, 1:5, seq(0,1,0.2))
# polynomial
# elison value is fixed at 0.1, but I left it so I didn't have to create a new function
svm_polynomial_inflamm = svm_classification(inflamm_resp_df, "polynomial", "Inflammatory_Responder", 
                                             "pred_Inflammatory_Responder", 0.035, 1:5, 0.1)
svm_polynomial_lung = svm_classification(lung_resp_df, "polynomial", "Lung_Responder", "pred_Lung_Responder",
                                             0.035, 1:5, 0.1)
svm_polynomial_inflamm_covariates = svm_classification(inflamm_resp_covariates_df, "polynomial", "Inflammatory_Responder", 
                                             "pred_Inflammatory_Responder", 0.035, 1:5, 0.1)
svm_polynomial_lung_covariates = svm_classification(lung_resp_covariates_df, "polynomial", "Lung_Responder", "pred_Lung_Responder",
                                             0.035, 1:5, 0.1)

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls > cases

Setting levels: control = 0, case = 1

Setting direction: controls > cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [14]:
# creating one combined df
svm_final_df = data.frame(rbind(svm_linear_inflamm, svm_linear_inflamm_covariates, svm_radial_inflamm, svm_radial_inflamm_covariates,
                                svm_polynomial_inflamm, svm_polynomial_inflamm_covariates, svm_linear_lung, svm_linear_lung_covariates,
                                svm_radial_lung, svm_radial_lung_covariates, svm_polynomial_lung, svm_polynomial_lung_covariates))
svm_final_df = data.frame(Model = rep(c("SVM Classification"), times = 12),
                          Response = c(rep(c("Inflammatory"), times = 6), rep(c("Lung"), times = 6)),
                          Covariates = rep(c("No", "Yes"), times = 6),
                      Kernel = rep(c(rep(c("linear"), times = 2), rep(c("radial"), times = 2), rep(c("polynomial"), times = 2)), times = 2),
            svm_final_df)
svm_final_df

Model,Response,Covariates,Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SVM Classification,Inflammatory,No,linear,0.4916667,0.6833333,0.3000000,NaN,0.4916667
SVM Classification,Inflammatory,Yes,linear,0.2666667,0.4333333,0.1000000,0.3500000,0.7666667
SVM Classification,Inflammatory,No,radial,0.5000000,1.0000000,0.0000000,0.6733333,0.5000000
SVM Classification,Inflammatory,Yes,radial,0.5000000,1.0000000,0.0000000,0.6733333,0.5000000
SVM Classification,Inflammatory,No,polynomial,0.5666667,0.9333333,0.2000000,0.7233333,0.5666667
SVM Classification,Inflammatory,Yes,polynomial,0.5666667,0.9333333,0.2000000,0.7233333,0.5666667
SVM Classification,Lung,No,linear,0.8333333,0.8333333,0.8333333,0.8666667,0.8333333
SVM Classification,Lung,Yes,linear,0.7333333,0.7333333,0.7333333,NaN,0.7333333
SVM Classification,Lung,No,radial,0.9333333,0.9333333,0.9333333,0.9333333,0.9333333


# K Nearest Neighbor

- seeks to choose the model that minimizes MSE or the difference with the true mean. The model is flexible, which will make it more likely to overfit the training dataset and be less generalizable to other datasets.
- works best for continuous predictors since it groups based on Euclidean distance (a different approach is needed for continuous and dichotomous predictors)

In [15]:
# model errors are calculated to assess the predictive accuracy of the model
knn_classification = function(dataset, outcome, pred_outcome){

    # setting seed for reproducibility
    set.seed(12)

    # splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) # 5 fold CV
    knn_errors = data.frame()

    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        # training algorithm 
        knn_model = train(as.formula(paste0(outcome, "~.")), data = data_train, method = 'knn',  
                          preProcess = c("center", "scale"))

        # assessing prediction
        data_test[[pred_outcome]] = predict(knn_model, newdata = data_test)
        matrix = confusionMatrix(data_test[[pred_outcome]], data_test[[outcome]])
        
        # calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        # extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)


        # adding values to df
        knn_errors = rbind(matrix_values, knn_errors)
    }

    
    # taking averages/sd 
    knn_errors = knn_errors %>%
        summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))

   return(knn_errors)
}

In [16]:
# calling fn
knn_values_inflamm_resp = knn_classification(inflamm_resp_df, "Inflammatory_Responder", "pred_Inflammatory_Responder")
knn_values_lung_resp = knn_classification(lung_resp_df, "Lung_Responder", "pred_Lung_Responder")
knn_values_inflamm_resp_covariates = knn_classification(inflamm_resp_covariates_df, "Inflammatory_Responder", "pred_Inflammatory_Responder")
knn_values_lung_resp_covariates = knn_classification(lung_resp_covariates_df, "Lung_Responder", "pred_Lung_Responder")

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Warning message:
“predictions failed for Resample15: k=5 Error in dimnames(x) <- dn : 
  length of 'dimnames' [2] not equal to array extent
”
Warning message:
“predictions failed for Resample15: k=7 Error in dimnames(x) <- dn : 
  length of 'dimnames' [2] not equal to array extent
”
Warning message:
“predictions failed for Resample15: k=9 Error in dimnames(x) <- dn : 
  length of 'dimnames' [2] not equal to array extent
”
Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”
Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in

Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Ethnicity1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Race2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Race2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Race2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Visit_Season2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Visit_Season2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have z

Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race2, Visit_Season2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race2, Visit_Season2”
Warning mes

Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Ethnicity1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Ethnicity1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Ethnicity1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Ethnicity1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Ethnicity1”
Wa

Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Race2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race2, Visit_Season2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race2, Visit_Season2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race2, Visi

...

Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Visit_Season2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Visit_Season2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Visit_Season2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Ethnicity1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Ethni

Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Ethnicity1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Ethnicity1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Ethnicity1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Race2, Ethnicity1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Race2, Ethnicity1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Race2, Ethnicity1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut =

Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Race2, Ethnicity1, Visit_Season2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Race2, Ethnicity1, Visit_Season2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Race2, Ethnicity1, Visit_Season2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Race2, Ethnicity1, Visit_Season2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Race2, Ethnicity1, Visit_Season2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Race2, E

Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race2, Ethnicity1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race2, Ethnicity1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race2, Ethnicity1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race2, Ethnicity1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race2, Ethnicity1”
Setting levels: control = 0, case = 1

Setting direction: controls < cases

Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race2”
Warning message in preProcess.default(thre

Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Race2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Race2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race1, Race2”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race2, Ethnicity1”
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
“These variables have zero variances: Race2, Ethnicit

In [17]:
#viewing results
knn_confusion_matrix = data.frame(Model = c(rep(c("KNN Classification"), times = 4)), 
                                 Response = c(rep(c("Inflammatory"), times = 2), rep(c("Lung"), times = 2)),
                                 Covariates = c(rep(c("No", "Yes"), times = 2)), Kernel = c(rep(c("NA"), times = 4)),
                                  rbind(knn_values_inflamm_resp, knn_values_inflamm_resp_covariates, 
                                        knn_values_lung_resp, knn_values_lung_resp_covariates))
knn_confusion_matrix

Model,Response,Covariates,Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
KNN Classification,Inflammatory,No,NA,0.5000000,0.0000000,1.0000000,NaN,0.5000000
KNN Classification,Inflammatory,Yes,NA,0.4666667,0.0000000,0.9333333,NaN,0.4666667
KNN Classification,Lung,No,NA,0.7833333,0.6333333,0.9333333,0.8000000,0.7833333
KNN Classification,Lung,Yes,NA,0.8500000,0.7666667,0.9333333,0.9333333,0.8500000


In [18]:
# combining into 1 df
final_df = rbind(rf_confusion_matrix, svm_final_df, knn_confusion_matrix) %>%
    arrange(Response, -Balanced.Accuracy)

final_df

Model,Response,Covariates,Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SVM Classification,Inflammatory,No,polynomial,0.5666667,0.9333333,0.2000000,0.7233333,0.5666667
SVM Classification,Inflammatory,Yes,polynomial,0.5666667,0.9333333,0.2000000,0.7233333,0.5666667
SVM Classification,Inflammatory,No,radial,0.5000000,1.0000000,0.0000000,0.6733333,0.5000000
SVM Classification,Inflammatory,Yes,radial,0.5000000,1.0000000,0.0000000,0.6733333,0.5000000
KNN Classification,Inflammatory,No,NA,0.5000000,0.0000000,1.0000000,NaN,0.5000000
SVM Classification,Inflammatory,No,linear,0.4916667,0.6833333,0.3000000,NaN,0.4916667
KNN Classification,Inflammatory,Yes,NA,0.4666667,0.0000000,0.9333333,NaN,0.4666667
RF Classification,Inflammatory,No,NA,0.4083333,0.8166667,0.0000000,0.6100000,0.5416667
RF Classification,Inflammatory,Yes,NA,0.3583333,0.7166667,0.0000000,0.5733333,0.5083333


In [19]:
#exporting
write.csv(final_df, paste0(Output,"/", cur_date, "_Ozone_Response_All_Model_Results.csv"), row.names = FALSE)